In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from datetime import date
import os

In [2]:
# headers -> Ctrl + shift + i (pagina de desarrollador) -> network Ctrl + f5 -> primera pestaña -> request headers
# desde "accept" ponerlo en dict 
headers = {
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-GB,en;q=0.9,en-US;q=0.8,ca;q=0.7,es;q=0.6,eo;q=0.5",
"cache-control": "no-cache",
"cookie": '''atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2268f673fe-430e-424d-9df0-0cd8699a34a7%22%2C%22options%22%3A%7B%22end%22%3A%222023-03-03T18%3A46%3A49.424Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-582065-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D; didomi_token=eyJ1c2VyX2lkIjoiMTdlYWM0ZmEtNzcxNC02MGMyLWJmODMtYWRjMGViMzFiYzE3IiwiY3JlYXRlZCI6IjIwMjItMDEtMzBUMTg6NDY6NTEuMTA0WiIsInVwZGF0ZWQiOiIyMDIyLTAxLTMwVDE4OjQ2OjUxLjEwNFoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzptaXhwYW5lbCIsImM6YWJ0YXN0eS1MTGtFQ0NqOCIsImM6aG90amFyIiwiYzp5YW5kZXhtZXRyaWNzIiwiYzpiZWFtZXItSDd0cjdIaXgiLCJjOmFwcHNmbHllci1HVVZQTHBZWSIsImM6dGVhbGl1bWNvLURWRENkOFpQIiwiYzppZGVhbGlzdGEtTHp0QmVxRTMiLCJjOmlkZWFsaXN0YS1mZVJFamUyYyJdfSwicHVycG9zZXMiOnsiZW5hYmxlZCI6WyJhbmFseXRpY3MtSHBCSnJySzciLCJnZW9sb2NhdGlvbl9kYXRhIl19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFGbUFDQUZrLkFBQUEifQ==; euconsent-v2=CPTmlIAPTmlIAAHABBENB_CoAP_AAAAAAAAAF5wBAAIAAtAC2AvMAAABAaADAAEEQyUAGAAIIhlIAMAAQRDIQAYAAgiGOgAwABBEMJABgACCIYyADAAEEQxUAGAAIIhg.f_gAAAAAAAAA; _gcl_au=1.1.8146483.1643568415; _fbp=fb.1.1643568415299.2070506742; afUserId=58c2a54c-4375-4eb4-baca-6a6e48c65725-p; AF_SYNC=1643568416416; askToSaveAlertPopUp=true; _hjSessionUser_250321=eyJpZCI6ImU0N2FkYjkyLTgyMDMtNWU0Mi1iMTRiLWIzMzQzMDE0NTM0NiIsImNyZWF0ZWQiOjE2NDM1Njg1MjgzMzYsImV4aXN0aW5nIjp0cnVlfQ==; userUUID=6eadbb28-4981-4c13-abda-fd7fadaec832; SESSION=f1f80941eb4fade2~bda569d4-9873-4a74-80a2-5fd0ae1a6c25; contactbda569d4-9873-4a74-80a2-5fd0ae1a6c25="{'email':null,'phone':null,'phonePrefix':null,'friendEmails':null,'name':null,'message':null,'message2Friends':null,'maxNumberContactsAllow':10,'defaultMessage':true}"; cookieSearch-1="/alquiler-viviendas/barcelona/les-corts/les-corts/:1643650811421"; sendbda569d4-9873-4a74-80a2-5fd0ae1a6c25="{'friendsEmail':null,'email':null,'message':null}"; _hjIncludedInSessionSample=1; _hjSession_250321=eyJpZCI6IjYyNTE2ZTJjLWNkNTYtNDRkZS05NmJkLWQ4ZjYxYjQwMGE1NCIsImNyZWF0ZWQiOjE2NDM2NTEzNjU5MTEsImluU2FtcGxlIjp0cnVlfQ==; _hjAbsoluteSessionInProgress=0; _hjCachedUserAttributes=eyJhdHRyaWJ1dGVzIjp7ImlkX3BhZ2VMYW5ndWFnZSI6ImVzIiwiaWRfdXNlclJvbGUiOiIifSwidXNlcklkIjpudWxsfQ==; ABTasty=uid=s43fcyc0hr2z8nkz&fst=1643568412317&pst=1643568412317&cst=1643650069991&ns=2&pvt=35&pvis=13&th=; ABTastySession=mrasn=&sen=12&lp=https%253A%252F%252Fwww.idealista.com%252F; utag_main=v_id:017eac27920f001f94d41f93932f05072002f06a00978$_sn:3$_se:17$_ss:0$_st:1643653573939$dc_visit:2$ses_id:1643650066499%3Bexp-session$_pn:13%3Bexp-session$_prevVtSource:directTraffic%3Bexp-1643653668218$_prevVtCampaignCode:%3Bexp-1643653668218$_prevVtDomainReferrer:%3Bexp-1643653668218$_prevVtSubdomaninReferrer:%3Bexp-1643653668218$_prevVtUrlReferrer:%3Bexp-1643653668218$_prevVtCampaignLinkName:%3Bexp-1643653668218$_prevVtCampaignName:%3Bexp-1643653668218$_prevVtRecommendationId:%3Bexp-1643653668218$_prevCompletePageName:12%3A%3Adetail%3A%3A%3A%3A%3A%3Ahome%3Bexp-1643655375478$_prevLevel2:12%3Bexp-1643655375478$_prevAdId:94476328%3Bexp-1643655375489$_prevAdOriginTypeRecommended:undefined%3Bexp-1643653668228$dc_event:5%3Bexp-session$dc_region:us-east-1%3Bexp-session; cto_bundle=5izo9l9XUkgzNEJIYU9ITGFNWU5xMWp4YWoxNCUyRnFJN2dHbFJTbWRCVlpldHRFbjF4aFFaSmFvWHZvV3B2WCUyRmx1MEhiMUNQbDlUQmlPckkwdW51VTdTZjltc255c0ZjRng0d1c4MXVFVURBdUFXdEYwaSUyQnhtSlowY3oybzE4TkxtY2RVWmR3ZHJvbVJvUko1MmQxT1FJczJvV2clM0QlM0Q; datadome=v475FOZILU7oISIcQM..EhoU-BUZiH-ZFK2rcgXnvpb4HM05afYuAyFXtJCwx2Wz7bUmxBeuhegZikjdshl3ZtuSvzBOApWI_EnGCatwDUdNRGdY_Ox0oK6H5CgXTyT''',
"pragma": "no-cache",
"referer": "https://www.idealista.com/inmueble/95249555/",
"sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": '"Windows"',
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "same-origin",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
    
}


# headers = {'User-agent': 'your bot 0.1'} # no funciona necesita datos reales?

### Funcion usar ids y devolver caracteristicas inmueble:

In [3]:
def get_ids(headers,cp):
    '''
    Function that returns all the flats ids of certain zip code with fixed headers request.
    The function return a list with the ids.
    
    headers: needed to obtain response 200 (success).
    cp: zip code.
    proxies: proxy to try.
    
    Returns list of ids.
    '''
    cp = cp
    x = 2 # 1
    ids = []
    
    try:
        
        # nueva linea tratando los casos donde hay solo una pagina de anuncios.
        
        url = f'https://www.idealista.com/buscar/alquiler-viviendas/{cp}/'
        r = requests.get(url, headers = headers)
        soup = bs(r.text,'lxml')
        
        articles = soup.find('main', {'class':'listing-items'}).find_all('article')
        
        for article in articles:
            id_piso = article.get('data-adid')  # get()  return the value of the key
            ids.append(id_piso)
        
#         time.sleep(random.randint(1,3)*random.random())
            time.sleep(random.randint(1,2)*random.random())
        
        
        while True:
            
            url = f'https://www.idealista.com/buscar/alquiler-viviendas/{cp}/pagina-{x}.htm'
            r = requests.get(url, headers = headers)
            soup = bs(r.text,'lxml')

            pag_actual = int(soup.find('main', {'class':'listing-items'}).find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)

            if x == pag_actual:
                articles = soup.find('main', {'class':'listing-items'}).find_all('article')
            else:
                break

            for article in articles:
                id_piso = article.get('data-adid')  # get()  return the value of the key
                ids.append(id_piso)

        #     id_pisos =  [article.get('data-adid') for article in articles]  # get()  return the value of the key
                time.sleep(random.randint(1,2)*random.random()) #
        #     ids.append(id_pisos)

            x += 1

    except:

        print(r)
        
    return ids
    

    

In [4]:
def obtain_data_from_ids(ids,headers,cp):
    '''
    Function that return all the data from each add id from previous function "get_ids".
    It returns the data in dictionary format, ready to be converted to dataframe.
    
    ids = list of ids.
    headers = to avoid errors, response 200 (success)
    cp = zip code of the ids
    
    Returns a dic with id as a key.
    '''
    
    dic = {}
    x=1
    

    for i in ids:
        print("aqui entra en el bucle")
        try:
            
            print("aqui esta en el try")
        
            url = f"https://www.idealista.com/inmueble/{i}/"
            
            print("id:",i)

            r = requests.get(url,headers=headers) # añadimos parametro headers
#             print(r)
            soup = bs(r.text,'lxml')  # formato mas legible
#             print(soup)
            # ubicacion
            ubi = soup.find("div",{"id":"headerMap"}).find_all("li",)  # lista
            
            ubicacion_full = [u.text.replace("\n","") for u in ubi] # ubicacion completa
            ubicacion_full = '||'.join(ubicacion_full) # string
    
            
            titulo = soup.find('h1').text.strip() 
#             comentario = soup.find("div",{"class":"comment"}).text.replace('\n','')
            localizacion = soup.find('span',{'class':'main-info__title-block'}).find("span").text
            # ubi table
            distrito2 = "".join(re.findall(r"Distrito[^|]*",ubicacion_full))
            barrio2 = "".join(re.findall(r"Barrio[^|]*",ubicacion_full))
            calle = ubi[0].text.replace("\n","")
            barrio = ubi[1].text.replace("\n","")
            distrito = ubi[2].text.replace("\n","")
            area = ubi[4].text.replace("\n","")
            
            precio = soup.find('div',{'class':'info-data'}).find('span').text.strip(' €/mes')
            
#             print("hasta aqui? precio")
            
            if "." in precio:
                precio = precio.replace('.','') # falta formato int

            precio_down = soup.find("span",{"class":"pricedown_price"})
            
#             print("hasta aqui? precio down")

            if precio_down != None :
                precio_down = precio_down.text.strip("\n").strip(" €").replace('.','')
            else:
                precio_down = precio
                #########
                
#             print("hasta aqui? precio down 2")
            
#             bug duplicate price down 
            
#             precio = soup.find('div',{'class':'info-data'}).find('span').text.strip(' €/mes')
#             if "." in precio:
#                 precio = precio.replace('.','') # falta formato int

#                 precio_down = soup.find("span",{"class":"pricedown_price"})
                
#             print("hasta aqui? precio down 3?")

#             if precio_down != None :
#                 precio_down = precio_down.text.strip("\n").strip(" €").replace('.','')
#             else:
#                 precio_down = precio
                
#             print("hasta aqui? precio down 4?")
#             print("hasta aqui?")
###############################
            # detalles basicos del piso habitaciones, baños, etc
            detalles = soup.find('section',{"id":"details"}).find("div",{"class":"details-property-feature-one"})
            # info sobre aire acondicionado, psicina, zonas verdes y consumo energetico
            detalles2 = soup.find('section',{"id":"details"}).find("div",{"class":"details-property-feature-two"})
            caract = [det.text.strip() for det in detalles.find_all("li")]

            s = ' '.join(caract).lower()

            metros = re.search(r'\d\d\d m²|\d\d m²',s).group().replace(' m²','')
            # util = re.search(r'\d\d m² útiles',s).group()
            hab =  "".join(re.findall(r'\d hab|sin habitación',s))
            wc =  "".join(re.findall(r'\d baño',s))
            terraza = "".join(re.findall(r'terraza',s))
            balcon = "".join(re.findall(r'balcón|balcon',s))
            estado = "".join(re.findall(r'segunda mano/buen estado',s))
            year = "".join(re.findall(r'\d\d\d\d',s))
            armarios = "".join(re.findall(r'armarios',s))
            cocina = "".join(re.findall(r'cocina equipada|cocina sin equipar',s))
            amu = "".join(re.findall(r'amueblada|amueblado|sin amueblar',s))
            # noAmu = re.findall(r'sin amueblar',s)
            planta = "".join(re.findall(r'planta \d\d|planta \d|bajo',s))
            calefac = "".join(re.findall(r'no dispone de calefacción|calefacción',s))
            ascn = "".join(re.findall(r'con ascensor|sin ascensor',s))
            aire = detalles2.find('ul').text.strip("\n").lower()
            if aire == "aire acondicionado":
                aire = 1
            else:
                aire = 0
            exterior = "".join(re.findall(r'exterior|interior',s))
            datalles2 = "".join([det.text.replace("\n","|").lower() for det  in detalles2.find_all('ul')]) # añado zonas verdes, psicina
            cp = cp
            actualizacion = soup.find('p',{"class":"date-update-text"}).text
            #     aire = "".join(re.findall(r'aire acondicionado',s))
            actualizacion2 = soup.find("p", {"class":"stats-text"}).text
            extract_day = date.today()
#             soup_save = soup
#             print("hasta aqui?")

#             print("aqui empieza el dic",titulo)
#             print("aqui empieza el dic",comentario)
#             print("aqui empieza el dic",localizacion)
#             print("aqui empieza el dic",ubicacion_full)
#             print("aqui empieza el dic",distrito2)
#             print("aqui empieza el dic",calle)
#             print("aqui empieza el dic",barrio)
#             print("aqui empieza el dic",distrito)
#             print("aqui empieza el dic",area)
#             print("aqui empieza el dic",precio)
#             print("aqui empieza el dic",precio_down)
#             print("aqui empieza el dic",metros)
#             print("aqui empieza el dic",hab)
#             print("aqui empieza el dic",wc)
#             print("aqui empieza el dic",terraza)
#             print("aqui empieza el dic",balcon)
#             print("aqui empieza el dic",estado)
#             print("aqui empieza el dic",year)
#             print("aqui empieza el dic",armarios)
#             print("aqui empieza el dic",cocina)
#             print("aqui empieza el dic",amu)
#             print("aqui empieza el dic",planta)
#             print("aqui empieza el dic",calefac)
#             print("aqui empieza el dic",ascn)
#             print("aqui empieza el dic",aire)
#             print("aqui empieza el dic",exterior)
#             print("aqui empieza el dic",datalles2)
#             print("aqui empieza el dic",cp)
#             print("aqui empieza el dic",actualizacion)
#             print("aqui empieza el dic",actualizacion2)
#             print("aqui empieza el dic",extract_day)

            dic[i] = [titulo,localizacion,ubicacion_full,distrito2,calle,barrio,barrio2,distrito,area,precio,precio_down,metros,hab,wc,terraza,balcon,estado,year,armarios,cocina,amu,planta,calefac,ascn,aire,exterior,datalles2,cp,actualizacion,actualizacion2,extract_day]
#             print(ubicacion_full)
#             print(dic[i])
#             print(precio)            
            len_ids = len(ids)
            print(r, "anuncio número:{}/{} cp: {}".format(x,len_ids,cp),sep="\n")
            
            x += 1
            
            time.sleep(random.randint(2,4)*random.random())
        
        except:
            print(">>>>>>>>>>>>>>>>>>>>>>>> error 404?, id ",i,r)
            time.sleep(random.randint(2,4)*random.random())
        
    return dic


In [5]:
def dic_to_df(dic):
    '''
    Transform dic from Idealista https://www.idealista.com/inmueble
    to a data frame.
    
    dic -> Dataframe
    '''
    colnames = ['name','zone','ubicacion_full','distrito2','calle','barrio','barrio2','distrito','area','price','price_before','square_mt','rooms','wc','terraza','balcon','estado',\
     'año','armarios','cocina','amueblado','planta','calef','asc','aire','exterior','datalles2','cp','actualizacion','actualizacion2','extract_day']
    df = pd.DataFrame.from_dict(dic, orient='index',columns=colnames)
    
    return df
    

In [6]:
def df_parse(df):
    '''
    Transform dataframe from Idealista https://www.idealista.com/inmueble
    to a clean data frame and convert the data types to int when possible.
    
    Dataframe -> Dataframe
    '''
    df['price'] = df.price.astype(int)
    df['price_before'] = df.price_before.astype(int)
    df['square_mt'] = df.square_mt.astype(int)
    try:
        df ['rooms']=df.rooms.apply(lambda x: x.replace('sin habitación','0'))
        df ['rooms']=df.rooms.apply(lambda x: x.replace(' hab','')).astype(int)
    except:
        pass
    try:
        df ['wc']=df.wc.apply(lambda x: x.replace(' baño','')).astype(int)
    #     df ['wc']=df.rooms.apply(lambda x: x.replace(' hab','')).astype(int)
    except:
        pass
    df['terraza'] = df.terraza.apply(lambda x:  x.replace('terraza','1') if x == 'terraza' else '0').astype(int)
    df['balcon'] = df.balcon.apply(lambda x:  x.replace('balcón','1') if x == 'balcón' else '0').astype(int)
    df['armarios'] = df.armarios.apply(lambda x:  x.replace('armarios','1') if x == 'armarios' else '0').astype(int)
    df['cocina'] = df.cocina.apply(lambda x:  x.replace('cocina equipada','1') if x == 'cocina equipada' else '0').astype(int)
    df['amueblado'] = df.amueblado.apply(lambda x:  x.replace('amueblado','1') if x == 'amueblado' else '0').astype(int)
    df['calef'] = df.calef.apply(lambda x:  x.replace('calefacción','1') if x == 'calefacción' else '0').astype(int)
    df['asc'] = df.asc.apply(lambda x:  x.replace('con ascensor','1') if x == 'con ascensor' else '0').astype(int)
    df['aire'] = df.aire.astype(int)
    df['exterior'] = df.exterior.apply(lambda x:  x.replace('exterior','1') if x == 'exterior' else '0').astype(int)
    try:
        df['planta'] = df.planta.apply(lambda x: x.replace('planta ',''))
        df['planta'] = df.planta.apply(lambda x: '0' if x == "" else x)
        df['planta'] = df.planta.apply(lambda x: '0' if x == "bajo" else x)
        df['planta'] = df.planta.astype(int)
    except:
        pass
    df['cp'] = df.cp.astype(str)
    
    return df


#### Prueba de una sola id

In [7]:
# # prueba con una sola id
# dic = {}
# cp = "08039" # barceloneta  ids
# i = "83328665"
# url = f"https://www.idealista.com/inmueble/{i}/"

# print("id:",i)

# r = requests.get(url,headers=headers) # añadimos parametro headers
# print("respuests: ",r)
# soup = bs(r.text,'lxml')  # formato mas legible

# titulo = soup.find('h1').text.strip() 
# # comentario = soup.find("div",{"class":"comment"}).text.replace('\n','')
# localizacion = soup.find('span',{'class':'main-info__title-block'}).find("span").text
# precio = soup.find('div',{'class':'info-data'}).find('span').text.strip(' €/mes')
# if "." in precio:
#     precio = precio.replace('.','') # falta formato int
    
# precio_down = soup.find("span",{"class":"pricedown_price"})

# if precio_down != None :
#     precio_down = precio_down.text.strip("\n").strip(" €").replace('.','')
# else:
#     precio_down = precio

# # detalles basicos del piso habitaciones, baños, etc
# detalles = soup.find('section',{"id":"details"}).find("div",{"class":"details-property-feature-one"})
# # info sobre aire acondicionado, psicina, zonas verdes y consumo energetico
# detalles2 = soup.find('section',{"id":"details"}).find("div",{"class":"details-property-feature-two"})
# caract = [det.text.strip() for det in detalles.find_all("li")]

# s = ' '.join(caract).lower()

# metros = re.search(r'\d\d\d m²|\d\d m²',s).group().replace(' m²','')
# # util = re.search(r'\d\d m² útiles',s).group()
# hab =  "".join(re.findall(r'\d hab|sin habitación',s))
# wc =  "".join(re.findall(r'\d baño',s))
# terraza = "".join(re.findall(r'terraza',s))
# balcon = "".join(re.findall(r'balcón|balcon',s))
# estado = "".join(re.findall(r'segunda mano/buen estado',s))
# year = "".join(re.findall(r'\d\d\d\d',s))
# armarios = "".join(re.findall(r'armarios',s))
# cocina = "".join(re.findall(r'cocina equipada|cocina sin equipar',s))
# amu = "".join(re.findall(r'amueblada|amueblado|sin amueblar',s))
# # noAmu = re.findall(r'sin amueblar',s)
# planta = "".join(re.findall(r'planta \d\d|planta \d|bajo',s))
# calefac = "".join(re.findall(r'no dispone de calefacción|calefacción',s))
# ascn = "".join(re.findall(r'con ascensor|sin ascensor',s))
# aire = detalles2.find('ul').text.strip("\n").lower()
# if aire == "aire acondicionado":
#     aire = 1
# else:
#     aire = 0
# exterior = "".join(re.findall(r'exterior|interior',s))
# datalles2 = "".join([det.text.replace("\n","|").lower() for det  in detalles2.find_all('ul')]) # añado zonas verdes, psicina
# cp = cp
# actualizacion = soup.find('p',{"class":"date-update-text"}).text
# #     aire = "".join(re.findall(r'aire acondicionado',s))
# actualizacion2 = soup.find("p", {"class":"stats-text"}).text
# extract_day = date.today()
# # soup_save = soup

# ubi = soup.find("div",{"id":"headerMap"}).find_all("li",)
# ubicacion_full = [u.text.replace("\n","") for u in ubi]
# ubicacion_full = '||'.join(ubicacion_full)

# distrito = "".join(re.findall(r"Distrito[^|]*",ubicacion_full))
# barrio = "".join(re.findall(r"Barrio[^|]*",ubicacion_full))

# barrio2 = "".join(re.findall(r"Barrio[^|]*",ubicacion_full))

# dic[i] = [titulo,localizacion,precio,precio_down,metros,hab,wc,terraza,balcon,estado,year,armarios,cocina,amu,planta,calefac,ascn,aire,exterior,cp,actualizacion,extract_day,barrio2]


# # print(dic)
# # print(soup)

# # var utag_data

# # print(soup.find('h1').text.strip()) 

# # soup.find_all('script')
# # print(soup.find('body class="" id=""')) 

# dic

In [8]:
# # obtain_data_from_ids(get_ids(headers,'08039'),headers,['08039'])
# obtain_data_from_ids(['99072099'],headers,['08039'])
# # obtain_data_from_ids algo esta mal aqui
# # get_ids(headers,'08039')

#### Lista de codigos postales

In [9]:
# cp = ["{:05d}".format(i) for i in range(8016,8017)]
# cp = ["{:05d}".format(i) for i in range(8001,8011)]
# cp = ["{:05d}".format(i) for i in range(8011,8021)]
# cp = ["{:05d}".format(i) for i in range(8021,8031)]
cp = ["{:05d}".format(i) for i in range(8031,8043)] # 8001 - 8010 , 8011 - 8021, 8021 - 8031
cp

['08031',
 '08032',
 '08033',
 '08034',
 '08035',
 '08036',
 '08037',
 '08038',
 '08039',
 '08040',
 '08041',
 '08042']

In [10]:
url_cp = "https://www.idealista.com/buscar/alquiler-viviendas/{}/".format(cp[0])

In [11]:
url_cp

'https://www.idealista.com/buscar/alquiler-viviendas/08031/'

In [12]:
# len(cp)
# cp

# os.getcwd()

parent_path = 'C:\\Users\\ggari\\Desktop\\Master_MESIO\\TFM\\srapper'

#### Definiendo un nuevo dataframe para juntar todos los datos.

In [13]:
def create_folder(parent_path):
    
    today = date.today()
    directory = "extraction_{}".format(today)
    
    list_files = os.listdir(parent_path)
#     path = os.path.join(parent_path, directory)
    
    if directory not in list_files:
        path = os.path.join(parent_path, directory)
        os.mkdir(path)
#         os.close(path)
        print("Directory '% s' created" % directory)
    else:
#         os.close(path)
        print("Directory '% s' already created" % directory)
    
    return directory
        

In [14]:
def call_scrapper(cp,parent_path):
    
    print("parent directory: ",os.getcwd())
    
    new_dir_name = create_folder(parent_path)
    
    os.chdir(".\\{}".format(new_dir_name))

    print("csv folder: ",os.getcwd())
    
    for ind, zip_code in enumerate(cp):
        print("buscando:",zip_code)
        id_from_cp = get_ids(headers, zip_code)
        time.sleep(random.randint(2,4)*random.random())
        dic_data = obtain_data_from_ids(id_from_cp,headers,cp = zip_code)
        time.sleep(random.randint(2,4)*random.random())
        df_data = df_parse(dic_to_df(dic_data))

        df_data.to_csv("data_{}.csv".format(zip_code),encoding = 'utf-8-sig',index_label= "id")
#         df_final = df_final.append(df_data)

    #     cp.pop(0) da problemas
    #     Para saber que cp han sido buscados se agregan a cp_buscados

#         cp_buscados.append(zip_code)

        print("guardado datos de:",zip_code)
        time.sleep(random.randint(2,4)*random.random()) 


In [15]:
def concat_csv(parent_path, folder=None):
    
    colnames = ['id','name','zone','ubicacion_full','calle','barrio','barrio2','distrito','area','price','price_before','square_mt','rooms','wc','terraza','balcon','estado',\
     'año','armarios','cocina','amueblado','planta','calef','asc','aire','exterior','datalles2','cp','actualizacion','actualizacion2','extract_day']
    df_final = pd.DataFrame(columns = colnames)
    
    today = date.today()
    
    if folder is None:
    
        directory = "extraction_{}".format(today)
        path = os.path.join(parent_path, directory)
        try:
            os.chdir(path)
            print("path exists")
        except:
            print("path does not exists")

        csv = os.listdir()
    
    if folder is not None:
        
#         today = date.today()
#         directory = "extraction_{}".format(today)
        path = os.path.join(parent_path, folder)
        try:
            os.chdir(path)
            print("path exists")
        except:
            print("path does not exists")
            
        csv = os.listdir()
        
    for i in csv:
        df = pd.read_csv(i)
        df_final = pd.concat([df_final,df],axis=0,ignore_index=True)
        
    df_final.to_csv("datos_scrapping_{}.csv".format(today),encoding = 'utf-8-sig',index=False)
    print("file '% s' created" % "datos_scrapping_{}.csv".format(today))
        
    


In [16]:
%%time

call_scrapper(cp,parent_path)

parent directory:  C:\Users\ggari\Desktop\Master_MESIO\TFM\srapper
Directory 'extraction_2023-01-12' already created
csv folder:  C:\Users\ggari\Desktop\Master_MESIO\TFM\srapper\extraction_2023-01-12
buscando: 08031
aqui entra en el bucle
aqui esta en el try
id: 90714941
<Response [200]>
anuncio número:1/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 99824916
<Response [200]>
anuncio número:2/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 100113369
<Response [200]>
anuncio número:3/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 99813671
<Response [200]>
anuncio número:4/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 100112635
<Response [200]>
anuncio número:5/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 100112402
<Response [200]>
anuncio número:6/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 100093357
<Response [200]>
anuncio número:7/39 cp: 08031
aqui entra en el bucle
aqui esta en el try
id: 88633969
<Re

<Response [200]>
anuncio número:38/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 37246015
<Response [200]>
anuncio número:39/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 100076422
<Response [200]>
anuncio número:40/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 100037707
<Response [200]>
anuncio número:41/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 95659056
<Response [200]>
anuncio número:42/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 100083027
<Response [200]>
anuncio número:43/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 99754999
<Response [200]>
anuncio número:44/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 98826955
<Response [200]>
anuncio número:45/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 98729162
<Response [200]>
anuncio número:46/72 cp: 08032
aqui entra en el bucle
aqui esta en el try
id: 99638359
<Response [200]>
anuncio número:47/72 cp: 08032
aqui entra en

aqui entra en el bucle
aqui esta en el try
id: 85182459
<Response [200]>
anuncio número:34/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 98813511
<Response [200]>
anuncio número:35/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 98710555
<Response [200]>
anuncio número:36/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 100073942
<Response [200]>
anuncio número:37/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 94098286
<Response [200]>
anuncio número:38/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 100098612
<Response [200]>
anuncio número:39/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 93758686
<Response [200]>
anuncio número:40/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99041260
<Response [200]>
anuncio número:41/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99885217
<Response [200]>
anuncio número:42/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 982691

aqui entra en el bucle
aqui esta en el try
id: 99620984
<Response [200]>
anuncio número:112/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 100050024
<Response [200]>
anuncio número:113/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99976810
<Response [200]>
anuncio número:114/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 36872829
<Response [200]>
anuncio número:115/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99899059
<Response [200]>
anuncio número:116/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99606419
<Response [200]>
anuncio número:117/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99479707
<Response [200]>
anuncio número:118/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 81224822
<Response [200]>
anuncio número:119/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 84581078
<Response [200]>
anuncio número:120/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id

<Response [200]>
anuncio número:189/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99594117
<Response [200]>
anuncio número:190/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 95850467
<Response [200]>
anuncio número:191/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99444391
<Response [200]>
anuncio número:192/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 98973896
<Response [200]>
anuncio número:193/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 98693574
<Response [200]>
anuncio número:194/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 99952634
<Response [200]>
anuncio número:195/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 36909658
<Response [200]>
anuncio número:196/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 87420735
<Response [200]>
anuncio número:197/198 cp: 08034
aqui entra en el bucle
aqui esta en el try
id: 35866657
<Response [200]>
anuncio número:198/198 cp: 08

aqui entra en el bucle
aqui esta en el try
id: 99924791
<Response [200]>
anuncio número:70/73 cp: 08035
aqui entra en el bucle
aqui esta en el try
id: 99869733
<Response [200]>
anuncio número:71/73 cp: 08035
aqui entra en el bucle
aqui esta en el try
id: 99624504
<Response [200]>
anuncio número:72/73 cp: 08035
aqui entra en el bucle
aqui esta en el try
id: 26480977
<Response [200]>
anuncio número:73/73 cp: 08035
guardado datos de: 08035
buscando: 08036
aqui entra en el bucle
aqui esta en el try
id: 100126957
<Response [200]>
anuncio número:1/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 100096259
<Response [200]>
anuncio número:2/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 83088517
<Response [200]>
anuncio número:3/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 90830587
<Response [200]>
anuncio número:4/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 100115322
<Response [200]>
anuncio número:5/109 cp: 08036
aqui entra en el buc

aqui entra en el bucle
aqui esta en el try
id: 100026113
<Response [200]>
anuncio número:75/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 32730467
<Response [200]>
anuncio número:76/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 38377392
<Response [200]>
anuncio número:77/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 26848828
<Response [200]>
anuncio número:78/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 98579722
<Response [200]>
anuncio número:79/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 98906531
<Response [200]>
anuncio número:80/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 100041313
<Response [200]>
anuncio número:81/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 100035326
<Response [200]>
anuncio número:82/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 96214623
<Response [200]>
anuncio número:83/109 cp: 08036
aqui entra en el bucle
aqui esta en el try
id: 99713

<Response [200]>
anuncio número:43/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 85739260
<Response [200]>
anuncio número:44/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 96287451
<Response [200]>
anuncio número:45/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 99975976
<Response [200]>
anuncio número:46/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 99261158
<Response [200]>
anuncio número:47/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 93330881
<Response [200]>
anuncio número:48/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 100061154
<Response [200]>
anuncio número:49/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 96106328
<Response [200]>
anuncio número:50/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 100049247
<Response [200]>
anuncio número:51/121 cp: 08037
aqui entra en el bucle
aqui esta en el try
id: 99931936
<Response [200]>
anuncio número:52/121 cp: 08037
aqui

<Response [200]>
anuncio número:121/121 cp: 08037
guardado datos de: 08037
buscando: 08038
aqui entra en el bucle
aqui esta en el try
id: 100122285
<Response [200]>
anuncio número:1/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 100082143
<Response [200]>
anuncio número:2/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 100120032
<Response [200]>
anuncio número:3/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 100114756
<Response [200]>
anuncio número:4/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 100103363
<Response [200]>
anuncio número:5/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 91888222
<Response [200]>
anuncio número:6/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 100099190
<Response [200]>
anuncio número:7/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 99958731
<Response [200]>
anuncio número:8/43 cp: 08038
aqui entra en el bucle
aqui esta en el try
id: 99209299
<Response [200]>
anunci

<Response [200]>
anuncio número:19/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 99985077
<Response [200]>
anuncio número:20/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 98928444
<Response [200]>
anuncio número:21/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 100089748
<Response [200]>
anuncio número:22/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 99900100
<Response [200]>
anuncio número:23/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 99923463
<Response [200]>
anuncio número:24/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 100058890
<Response [200]>
anuncio número:25/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 98597085
<Response [200]>
anuncio número:26/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 100038633
<Response [200]>
anuncio número:27/46 cp: 08041
aqui entra en el bucle
aqui esta en el try
id: 97982947
<Response [200]>
anuncio número:28/46 cp: 08041
aqui entra en

In [17]:
concat_csv(parent_path,folder="extraction_2023-01-12")
# parent_path

path exists
file 'datos_scrapping_2023-01-13.csv' created
